# NT + ESM-2 ZERO-SHOT Ver


### 1) Split parquet into train/val/test (chr20/21 as test)


In [ ]:
from split_data import main as split_main
from config import RAW_PARQUET
split_main(RAW_PARQUET)

### 2) Precompute embeddings (NT + ESM-2)


In [ ]:
%env TOKENIZERS_PARALLELISM=false
from precompute_embeddings import main as emb_main
emb_main()

### 3) Train fusion classifier

Để theo dõi tiến độ train trong tensorboard:

- Mở terminal -> nhập: tensorboard --logdir runs -> truy cập vào link được in ra


In [ ]:
from train import train
from config import LR, EPOCHS, BATCH_SIZE, PATIENCE, DROPOUT, SEED
import argparse

parser = argparse.ArgumentParser(description="Train fusion classifier with enhanced features.")
parser.add_argument("--lr", type=float, default=LR, help="Learning rate")
parser.add_argument("--epochs", type=int, default=EPOCHS)
parser.add_argument("--batch_size", type=int, default=BATCH_SIZE)
parser.add_argument("--patience", type=int, default=PATIENCE)
parser.add_argument("--dropout", type=float, default=DROPOUT)
parser.add_argument("--seed", type=int, default=SEED)
parser.add_argument("--log_dir", type=str, default="runs/experiment_1", help="TensorBoard log directory")

args = parser.parse_args([])

result = train(args)
result

# NT + ESM-2 FINE-TUNED Ver
